In [ ]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
from vamp42 import VAMP42

In [ ]:
def draw_line_segment(X, Y, normal, intercept, label=None):
    'Convenience fucnction to plot the 0.5 level set of the sigmoid. '
    maxx=max(np.max(X[:, 0]), np.max(Y[:, 0])) 
    minx=min(np.min(X[:, 0]), np.min(Y[:, 0]))
    maxy=max(np.max(X[:, 1]), np.max(Y[:, 1])) 
    miny=min(np.min(X[:, 1]), np.min(Y[:, 1]))    
    points = []
    xcut = -(normal[1]*maxy+intercept)/normal[0]
    if minx <= xcut <= maxx:
        points.append((xcut, maxy))
    xcut = -(normal[1]*miny+intercept)/normal[0]
    if minx <= xcut <= maxx:
        points.append((xcut, miny))
    ycut = -(normal[0]*maxx+intercept)/normal[1]
    if miny <= ycut <= maxy:
        points.append((maxx, ycut))
    ycut = -(normal[0]*minx+intercept)/normal[1]
    if miny <= ycut <= maxy:
        points.append((minx, ycut))
    points = np.array(points)
    plt.plot(points[:, 0], points[:, 1], label=label, color='orange', lw=3)

In [ ]:
dim = 2
X = np.vstack((np.random.randn(100, dim) + 4*np.ones(dim), 
               np.random.randn(100, dim)))
Y = X + np.random.randn(200, dim)

In [ ]:
vamp = VAMP42(X, Y, init='linear').run(steps=0)
plt.scatter(X[:, 0], X[:, 1], c=vamp.f(X))
plt.scatter(Y[:, 0], Y[:, 1], c=vamp.f(Y))
for x,y in zip(X, Y):
    plt.plot([x[0], y[0]], [x[1], y[1]], '-', c='gray')
normal, intercept, _ = vamp.ext_hnf
draw_line_segment(X, Y, normal, intercept, label='TICA')
plt.legend()

In [ ]:
@interact
def show(steps=(0,40,1)):
    vamp = VAMP42(X, Y, init=np.array([ 0.03572001, -0.09408172,  0.06941734]))
    vamp.run(steps=steps)
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], c=vamp.f(X))
    plt.scatter(Y[:, 0], Y[:, 1], c=vamp.f(Y))
    for x,y in zip(X, Y):
        plt.plot([x[0], y[0]], [x[1], y[1]], '-', c='gray')
    normal, intercept, _ = vamp.ext_hnf
    draw_line_segment(X, Y, normal, intercept, label='VAMP')
    plt.text(-3, 7, '%0.3f' % vamp.eigenvalue)
    plt.clim((0,1))
    plt.colorbar()

In [ ]:
%%script true
for steps in range(40):
    vamp = VAMP42(X, Y, init=np.array([ 0.03572001, -0.09408172,  0.06941734]))
    vamp.run(steps=steps)
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1], c=vamp.f(X))
    plt.scatter(Y[:, 0], Y[:, 1], c=vamp.f(Y))
    for x,y in zip(X, Y):
        plt.plot([x[0], y[0]], [x[1], y[1]], '-', c='gray')
    normal, intercept, steepness = vamp.ext_hnf
    draw_line_segment(X, Y, normal, intercept, label='VAMP')
    plt.text(-3, 7, '%0.3f' % vamp.eigenvalue)
    plt.clim((0,1))
    plt.colorbar()
    plt.savefig('imgs/%03d.png'%steps, dpi=200)

In [ ]:
#!(cd imgs; ffmpeg -y -r 5 -f image2 -i %03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p vamp.mp4)